## Construção de uma calculadora de renda fixa em Python - NTN-F


Este projeto tem o objetivo de elaborar uma calculadora de NTN-F (Notas do Tesouro Nacional série F), título do Tesouro Direto com taxa prefixada e com juros semestrais).

# Projeto

In [54]:
# Bibliotecas utilizadas
import pandas as pd
from datetime import datetime, date
import numpy as np
from dateutil.relativedelta import relativedelta

**Obs.**

**Na última célula do projeto consta a versao das bibliotecas utilizadas (requirements).**

In [55]:
# função para truncar as casa decimais
def truncar(num,n):
    numeros = str(num)
    
    for x in range(len(numeros)):
        if numeros[x] == '.':
            try:
                return float(numeros[:x+n+1])
            except:
                return float(numeros)      
            
    return float(numeros)

In [56]:
#teste função truncar
truncar(6.5578642, 4)


6.5578

In [57]:
def data_feriados ():
    # função que atribui a planilha de feriados do site da ambima em lista com valores no formato dd/mm/AAAA
    
    # convertendo em dataframe o excel buscado do site ambima
    df = pd.read_excel('https://www.anbima.com.br/feriados/arqs/feriados_nacionais.xls')
    
    # removendo as colunas dia semana e feriado para ter somente a data na lista
    df.drop(['Dia da Semana', 'Feriado'], axis = 1, inplace = True)
    
    # lista que transforma a data em string atravs do map e usando o filter para converter somente os valores que estão como datetime
    lista_feriados = list(map(lambda x: x.strftime("%d/%m/%Y"),filter(lambda x: type(x) is datetime, df['Data'])))
    
    #retorna a lista
    return lista_feriados

In [58]:
# teste função feriados

lista_feriados = data_feriados()

# exibindo a lista a cada 10 elementos
print('Lista de feriados ambima:',lista_feriados[::10])

Lista de feriados ambima: ['01/01/2001', '15/11/2001', '12/10/2002', '19/06/2003', '21/04/2004', '08/02/2005', '01/01/2006', '15/11/2006', '12/10/2007', '22/05/2008', '21/04/2009', '16/02/2010', '01/01/2011', '15/11/2011', '12/10/2012', '30/05/2013', '21/04/2014', '17/02/2015', '01/01/2016', '15/11/2016', '12/10/2017', '31/05/2018', '21/04/2019', '25/02/2020', '01/01/2021', '15/11/2021', '12/10/2022', '08/06/2023', '21/04/2024', '04/03/2025', '01/01/2026', '15/11/2026', '12/10/2027', '15/06/2028', '21/04/2029', '05/03/2030', '01/01/2031', '15/11/2031', '12/10/2032', '16/06/2033', '21/04/2034', '06/02/2035', '01/01/2036', '15/11/2036', '12/10/2037', '24/06/2038', '21/04/2039', '14/02/2040', '01/01/2041', '15/11/2041', '12/10/2042', '28/05/2043', '21/04/2044', '21/02/2045', '01/01/2046', '15/11/2046', '12/10/2047', '04/06/2048', '21/04/2049', '22/02/2050', '01/01/2051', '15/11/2051', '12/10/2052', '05/06/2053', '21/04/2054', '02/03/2055', '01/01/2056', '15/11/2056', '12/10/2057', '13/06/

Visando uma melhor implementação, foi utilizado a orientação a objetos para reaproveitamento de códigos e torná-los mais limpos, e também para possível ampliação a demais títulos.
Assim, foi pensado em 3 classes: Prazo (para calculo de prazo anual e as datas dos cupons), PreçoUnitario (calculo do preço unitário e taxa anual) e Titulos_NTNF (calculo dos pagamentos e exibição do fluxo de caixa Ntn-F). 

In [59]:
class Prazo():
    
    def __init__(self, dt_ini, dt_fim, convencao=252):
    # método construtor com as datas a convencao de dias e atribui a função com feriados ambima
        
        self.dt_ini = dt_ini
        self.dt_fim = dt_fim
        self.feriados = data_feriados()
        
        if convencao != 252 and convencao != 360:
            return print('Convenção deve ser 252 ou 360 dias!')
        
        self.convencao = convencao
        
  
    
    def calculo_prazo_anual (self):
    # calcula os dias dos cupons
        

        # converte a string em objeto data para o cálculo     
        dt_ini = datetime.strptime(self.dt_ini, '%d/%m/%Y')
        
        dt_fim = datetime.strptime(self.dt_fim, '%d/%m/%Y')

        # numero de cupons conforme a data final e inicial        
        n_cupom = int(((dt_fim.year - dt_ini.year) *12 + (dt_fim.month - dt_ini.month))/6)

        # variavel que no inicio será a data inical acrescentando 6 meses a cada loop
        dt_cupom = dt_ini

        # lista para receber os dias 'anualizados'
        dias_cupons = []
        

        # convertendo os valores da lista de feriados em date para utilizar na função busday a seguir
        feriados = list(map(lambda x: datetime.strptime(x, '%d/%m/%Y').date(), self.feriados))

        # iterando conforme quantidade de cupons
        for cupom in range (int(n_cupom+1)):

            # utilizando condicional para cálculo conforme a convencao
            
            # Calculos para convenção = 252
            ###############################


            if self.convencao == 252:

                # dias uteis da data inicial até o cupom retirando feriados da ambima
                dias_uteis = np.busday_count(dt_cupom.date(), dt_fim.date(), holidays = feriados)

                # a cada loop a data de cupom acrescenta 6 meses
                dt_cupom = dt_cupom + relativedelta(months = 6)

                dias_totais = dias_uteis/self.convencao

                dias_cupons.append(truncar(dias_totais, 14))



            # Calculo para convenção = 360
            ##############################
            else:

                # calcula dias corridos
                dias_corridos = abs((self.dt_fim - dt_cupom).days)
 
                dt_cupom = dt_cupom + relativedelta(months = 6)

                dias_totais = dias_corridos/self.convencao

                dias_cupons.append(truncar(dias_totais, 14))    


        #retorna os dias uteis conforme convençao
        return sorted(dias_cupons)
    
    

    def constroi_fluxo(self, frequencia = 6):
        # função que retorna lista das datas dos cupons

        dt_fim = datetime.strptime(self.dt_fim, '%d/%m/%Y')
        dt_ini = datetime.strptime(self.dt_ini, '%d/%m/%Y')
        
        n_cupom = int(((dt_fim.year - dt_ini.year) *12 + (dt_fim.month - dt_ini.month))/frequencia)

        # lista para receber as datas 
        datas_cupons= []

        # inserindo a data de vencimento que paga o último cupom (mais o principal)
        datas_cupons.append(dt_fim)
        
        # variável que inicialmente é a ultima data e a cada loop desconta a frequencia
        dt_regressiva = dt_fim
        
        for x in range (round(n_cupom)):

            dt_cupons = dt_regressiva - relativedelta(months = frequencia)
            datas_cupons.append(dt_cupons)
            dt_regressiva = dt_cupons

        # retorna lista de datas ordenando de forma crescente
        return sorted(datas_cupons)

In [60]:
# teste classe Prazo com prazos anualizados e datas
prazos = Prazo('14/02/2022','01/01/2031', 252)
print('\033[1m Dias uteis (prazo anual):\033[0m \n', prazos.calculo_prazo_anual(),'\n\n', '\033[1m Datas: \033[0m \n',
      list(map(lambda x: datetime.strftime(x, "%d/%m/%Y"), prazos.constroi_fluxo())), sep='')

 Dias uteis (prazo anual): 
[0.38888888888888, 0.88095238095238, 1.38095238095238, 1.88095238095238, 2.37301587301587, 2.86507936507936, 3.36507936507936, 3.86904761904761, 4.36111111111111, 4.84920634920634, 5.36507936507936, 5.8531746031746, 6.36507936507936, 6.86904761904761, 7.36111111111111, 7.84920634920634, 8.3531746031746, 8.84920634920635]

 Datas:  
['01/07/2022', '01/01/2023', '01/07/2023', '01/01/2024', '01/07/2024', '01/01/2025', '01/07/2025', '01/01/2026', '01/07/2026', '01/01/2027', '01/07/2027', '01/01/2028', '01/07/2028', '01/01/2029', '01/07/2029', '01/01/2030', '01/07/2030', '01/01/2031']


In [61]:
class PrecoUnitario(Prazo):
# classe que herda os atributos e métodos de Prazo para calculo do pu e taxa anual
    
    def __init__(self, valor_face,taxa_anual, dt_ini, dt_fim, convencao=252 ):
        
        self.valor_face = valor_face
        self.taxa_anual = taxa_anual
        
        
        super().__init__(dt_ini, dt_fim, convencao)
        
    def calcula_pu(self):
        
        # recebe os dias (prazos) pelo método da classe mãe
        prazo_anual = self.calculo_prazo_anual()
        
        # recebe a taxa anual pelo atributo da classe mãe
        taxa_anual = self.taxa_anual

        # tratando a taxa caso esteja em percentual
        if taxa_anual >1:
            taxa_anual = taxa_anual/100

        # variavel para receber a soma a cada calculo que será feito no loop
        soma = 0 


        # iterando no tamanho da lista com os prazos menos 1 
        # para que o último seja calculado com o valor de face mais o último cupom separado
        for x in range (len(prazo_anual)-1):

            # calculo do valor presente de cada cupom
            pu = (self.valor_face* ((1.1**0.5) - 1))/(1+(taxa_anual))**(prazo_anual[x])

            # incrementando a soma a cada loop
            soma = soma + pu

        # valor presente do pagamento no vencimento
        soma= soma + (self.valor_face* (1.1**0.5))/(1+(taxa_anual))**(prazo_anual[-1])

        return truncar(soma,6)

    def calcula_taxa_anual(self):
        
        # recebe o valor presente
        pu = self.calcula_pu()
        
        # recebe os prazos
        prazo_anual = self.calculo_prazo_anual()
        
        # dias totais, no caso o ultimo elemento da lista do prazo
        du_ano = prazo_anual[-1]

        # calculando o valor futuro, os cupons mais o principal
        self.valor_futuro =( len(prazo_anual) * (self.valor_face* ((1.1**0.5) - 1)) ) + self.valor_face  


        taxa_anual = (self.valor_futuro/pu)**(1/du_ano) -1

        return taxa_anual

In [62]:
# teste classe preço unitario
titulo = PrecoUnitario(1000, 0.1157,'14/02/2022','01/01/2031')

In [63]:
print('\033[1m Taxa anual calculada: \033[0m',titulo.calcula_taxa_anual(), '\n',
      '\033[1m Preço Unitario: \033[0m',titulo.calcula_pu(),'\n',' \033[1m Datas Cupons: \033[0m \n',
      list(map(lambda x: datetime.strftime(x, '%d/%m/%Y'), titulo.constroi_fluxo())), sep='')

 Taxa anual calculada: 0.08255567336277125
 Preço Unitario: 931.038447
  Datas Cupons:  
['01/07/2022', '01/01/2023', '01/07/2023', '01/01/2024', '01/07/2024', '01/01/2025', '01/07/2025', '01/01/2026', '01/07/2026', '01/01/2027', '01/07/2027', '01/01/2028', '01/07/2028', '01/01/2029', '01/07/2029', '01/01/2030', '01/07/2030', '01/01/2031']


In [68]:
class Ntnf(PrecoUnitario):
# calculo dos pagamentos e exibição fluxo de caixa, herda os dados de PrecoUnitario


    def __init__(self, valor_face,taxa_anual, dt_ini, dt_fim, convencao=252):
        super().__init__(valor_face,taxa_anual, dt_ini, dt_fim, convencao)
        
    
    def calcula_pu_ntnf(self):
        
        # recebe prazos herdados da classe Prazo
        prazo_anual = self.calculo_prazo_anual()
        
        # recebe preço unitario da classe PrecoUnitario
        pu = self.calcula_pu()
        
        # datas da classe Prazo
        datas = self.constroi_fluxo()
        
        # lista para receber os pagamentos a cada loop
        vfs = []
        
        # lista para receber os valores presentes de cada pagamento
        pus = []
        
        # lista fator de atualização
        fatores = []
        
        taxa_anual = self.taxa_anual


        for x in range (len(prazo_anual)-1):

            # calculo do valor presente de cada cupom
            valor_presente = (1000 *((1.1**0.5) - 1))/(1+(taxa_anual))**(prazo_anual[x]) 
            
            fator = (1+(taxa_anual))**(prazo_anual[x])
            
            # valor dos cupons
            cupons = (self.valor_face* ((1.1**0.5) - 1)) 

            pus.append(valor_presente)
            fatores.append(fator)
            vfs.append(cupons)
        
        # valor presente do último pagamento (vencimento) cupom e  valor de face
        pus_ultimo = (self.valor_face * (1.1**0.5))/(1+(taxa_anual))**(prazo_anual[-1])
        
        # valor futuro do último pagamento
        valor_f_ultimo =  (self.valor_face* ((1.1**0.5) - 1))  + self.valor_face 
        
        fator_ultimo = (1+(taxa_anual))**(prazo_anual[-1])
        
                
        pus.append(pus_ultimo)
        
        fatores.append(fator_ultimo)
        
        vfs.append(valor_f_ultimo)
        
        # convertendo os dias anualizados do prazo anual em dias conforme convencao
        dus_diario = list(map(lambda x: round(x *self.convencao) , prazo_anual))
        
        # construindo dataframe
        df = pd.DataFrame(list(zip(datas, vfs, dus_diario, fatores,pus)),
                          columns = ['Datas', 'Fluxo de Caixa (Cupons)', 'Dias úteis', 'Fator','Valor Presente'])
        
        # exibe em formato DataFrame
        display(df)

        return pu

In [69]:

titulo_ntnf = Ntnf(1000,0.1148,'18/02/2022','01/01/2031',252)

In [70]:
titulo_ntnf.calcula_taxa_anual()

0.08194861797177322

In [71]:
titulo_ntnf.calcula_pu_ntnf()

,Datas,Fluxo de Caixa (Cupons),Dias úteis,Fator,Valor Presente
0,2022-07-01,48.808848,95,1.041820,46.849618
1,2023-01-01,48.808848,220,1.099521,44.390993
2,2023-07-01,48.808848,344,1.159919,42.079538
3,2024-01-01,48.808848,471,1.225218,39.836869
4,2024-07-01,48.808848,594,1.291963,37.778837
5,2025-01-01,48.808848,718,1.362931,35.811679
6,2025-07-01,48.808848,846,1.440280,33.888443
7,2026-01-01,48.808848,972,1.520707,32.096165
8,2026-07-01,48.808848,1097,1.604932,30.411788
9,2027-01-01,48.808848,1222,1.693822,28.815805


936.841841

## Ambiente de bibliotecas do projeto realizado (requirements):

In [ ]:
'''
absl-py==1.0.0
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-client==1.7.2
anaconda-navigator==2.0.4
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1610472525955/work
anyio @ file:///C:/ci/anyio_1620153418380/work/dist
appdirs==1.4.4
argh==0.26.2
argon2-cffi @ file:///C:/ci/argon2-cffi_1613037959010/work
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astroid @ file:///C:/ci/astroid_1613501047216/work
astropy @ file:///C:/ci/astropy_1617745647203/work
astunparse==1.6.3
async-generator @ file:///home/ktietz/src/ci/async_generator_1611927993394/work
atomicwrites==1.4.0
attrs @ file:///tmp/build/80754af9/attrs_1604765588209/work
autopep8 @ file:///tmp/build/80754af9/autopep8_1615918855173/work
Babel @ file:///tmp/build/80754af9/babel_1607110387436/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
backports.functools-lru-cache @ file:///tmp/build/80754af9/backports.functools_lru_cache_1618170165463/work
backports.shutil-get-terminal-size @ file:///tmp/build/80754af9/backports.shutil_get_terminal_size_1608222128777/work
backports.tempfile @ file:///home/linux1/recipes/ci/backports.tempfile_1610991236607/work
backports.weakref==1.0.post1
bcrypt @ file:///C:/ci/bcrypt_1597936263757/work
beautifulsoup4 @ file:///home/linux1/recipes/ci/beautifulsoup4_1610988766420/work
bitarray @ file:///C:/ci/bitarray_1618435038389/work
bkcharts==0.2
black==19.10b0
bleach @ file:///tmp/build/80754af9/bleach_1612211392645/work
bokeh @ file:///C:/ci/bokeh_1620784067744/work
Boruta==0.3
boto==2.49.0
Bottleneck==1.3.2
brotlipy==0.7.0
cachetools==4.2.4
certifi==2020.12.5
cffi @ file:///C:/ci/cffi_1613247279197/work
chardet @ file:///C:/ci/chardet_1607690654534/work
click @ file:///home/linux1/recipes/ci/click_1610990599742/work
cloudpickle @ file:///tmp/build/80754af9/cloudpickle_1598884132938/work
clyent==1.2.2
colorama @ file:///tmp/build/80754af9/colorama_1607707115595/work
comtypes==1.1.9
conda==4.10.3
conda-build==3.21.4
conda-content-trust @ file:///tmp/build/80754af9/conda-content-trust_1617045594566/work
conda-package-handling @ file:///C:/ci/conda-package-handling_1618262320430/work
conda-repo-cli @ file:///tmp/build/80754af9/conda-repo-cli_1620168426516/work
conda-token @ file:///tmp/build/80754af9/conda-token_1620076980546/work
conda-verify==3.4.2
contextlib2==0.6.0.post1
cryptography @ file:///C:/ci/cryptography_1616769344312/work
cycler==0.10.0
Cython @ file:///C:/ci/cython_1618435363327/work
cytoolz==0.11.0
dask @ file:///tmp/build/80754af9/dask-core_1617390489108/work
decorator @ file:///tmp/build/80754af9/decorator_1617916966915/work
defusedxml @ file:///tmp/build/80754af9/defusedxml_1615228127516/work
diff-match-patch @ file:///tmp/build/80754af9/diff-match-patch_1594828741838/work
distributed @ file:///C:/ci/distributed_1617384289923/work
docutils @ file:///C:/ci/docutils_1617481617511/work
entrypoints==0.3
et-xmlfile==1.0.1
fastcache==1.1.0
filelock @ file:///home/linux1/recipes/ci/filelock_1610993975404/work
flake8 @ file:///tmp/build/80754af9/flake8_1615834841867/work
Flask @ file:///home/ktietz/src/ci/flask_1611932660458/work
flatbuffers==2.0
fsspec @ file:///tmp/build/80754af9/fsspec_1617959894824/work
future==0.18.2
gast==0.4.0
gevent @ file:///C:/ci/gevent_1616773090559/work
glob2 @ file:///home/linux1/recipes/ci/glob2_1610991677669/work
google-auth==2.3.3
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
greenlet @ file:///C:/ci/greenlet_1611958565931/work
grpcio==1.43.0
h5py==2.10.0
HeapDict==1.0.1
html5lib @ file:///tmp/build/80754af9/html5lib_1593446221756/work
idna @ file:///home/linux1/recipes/ci/idna_1610986105248/work
imagecodecs @ file:///C:/ci/imagecodecs_1617996768495/work
imageio @ file:///tmp/build/80754af9/imageio_1617700267927/work
imagesize @ file:///home/ktietz/src/ci/imagesize_1611921604382/work
importlib-metadata==4.10.0
iniconfig @ file:///home/linux1/recipes/ci/iniconfig_1610983019677/work
intervaltree @ file:///tmp/build/80754af9/intervaltree_1598376443606/work
ipykernel @ file:///C:/ci/ipykernel_1596190155316/work/dist/ipykernel-5.3.4-py3-none-any.whl
ipython @ file:///C:/ci/ipython_1617121002983/work
ipython-genutils @ file:///tmp/build/80754af9/ipython_genutils_1606773439826/work
ipywidgets @ file:///tmp/build/80754af9/ipywidgets_1610481889018/work
isort @ file:///tmp/build/80754af9/isort_1616355431277/work
itsdangerous @ file:///home/ktietz/src/ci/itsdangerous_1611932585308/work
jdcal==1.4.1
jedi @ file:///C:/ci/jedi_1606914528444/work
Jinja2 @ file:///tmp/build/80754af9/jinja2_1612213139570/work
joblib @ file:///tmp/build/80754af9/joblib_1613502643832/work
json5==0.9.5
jsonschema @ file:///tmp/build/80754af9/jsonschema_1602607155483/work
jupyter==1.0.0
jupyter-client @ file:///tmp/build/80754af9/jupyter_client_1616770841739/work
jupyter-console @ file:///tmp/build/80754af9/jupyter_console_1616615302928/work
jupyter-core @ file:///C:/ci/jupyter_core_1612213356021/work
jupyter-packaging @ file:///tmp/build/80754af9/jupyter-packaging_1613502826984/work
jupyter-server @ file:///C:/ci/jupyter_server_1616084298403/work
jupyterlab @ file:///tmp/build/80754af9/jupyterlab_1619133235951/work
jupyterlab-pygments @ file:///tmp/build/80754af9/jupyterlab_pygments_1601490720602/work
jupyterlab-server @ file:///tmp/build/80754af9/jupyterlab_server_1617134334258/work
jupyterlab-widgets @ file:///tmp/build/80754af9/jupyterlab_widgets_1609884341231/work
keras==2.7.0
Keras-Preprocessing==1.1.2
keyring @ file:///C:/ci/keyring_1614616910860/work
kiwisolver @ file:///C:/ci/kiwisolver_1612282606037/work
lazy-object-proxy @ file:///C:/ci/lazy-object-proxy_1616529307648/work
libarchive-c @ file:///tmp/build/80754af9/python-libarchive-c_1617780486945/work
libclang==12.0.0
lightgbm==3.3.1
llvmlite==0.36.0
locket==0.2.1
lxml @ file:///C:/ci/lxml_1616443455957/work
Markdown==3.3.6
MarkupSafe==1.1.1
matplotlib @ file:///C:/ci/matplotlib-suite_1613408055530/work
mccabe==0.6.1
menuinst==1.4.16
mistune==0.8.4
mkl-fft==1.3.0
mkl-random @ file:///C:/ci/mkl_random_1618854156666/work
mkl-service==2.3.0
mlxtend==0.19.0
mock @ file:///tmp/build/80754af9/mock_1607622725907/work
more-itertools @ file:///tmp/build/80754af9/more-itertools_1613676688952/work
mpmath==1.2.1
msgpack @ file:///C:/ci/msgpack-python_1612287368835/work
multipledispatch==0.6.0
mypy-extensions==0.4.3
mysql-connector==2.2.9
navigator-updater==0.2.1
nbclassic @ file:///tmp/build/80754af9/nbclassic_1616085367084/work
nbclient @ file:///tmp/build/80754af9/nbclient_1614364831625/work
nbconvert @ file:///C:/ci/nbconvert_1601914925608/work
nbformat @ file:///tmp/build/80754af9/nbformat_1617383369282/work
nest-asyncio @ file:///tmp/build/80754af9/nest-asyncio_1613680548246/work
networkx @ file:///tmp/build/80754af9/networkx_1598376031484/work
nltk @ file:///tmp/build/80754af9/nltk_1618327084230/work
nose @ file:///tmp/build/80754af9/nose_1606773131901/work
notebook @ file:///C:/ci/notebook_1616443715883/work
numba @ file:///C:/ci/numba_1616774458845/work
numexpr @ file:///C:/ci/numexpr_1618856738664/work
numpy @ file:///C:/ci/numpy_and_numpy_base_1618497418457/work
numpydoc @ file:///tmp/build/80754af9/numpydoc_1605117425582/work
oauthlib==3.1.1
olefile==0.46
openpyxl @ file:///tmp/build/80754af9/openpyxl_1615411699337/work
opt-einsum==3.3.0
packaging @ file:///tmp/build/80754af9/packaging_1611952188834/work
pandas==1.3.5
pandocfilters @ file:///C:/ci/pandocfilters_1605102497129/work
paramiko @ file:///tmp/build/80754af9/paramiko_1598886428689/work
parso==0.7.0
partd @ file:///tmp/build/80754af9/partd_1618000087440/work
path @ file:///C:/ci/path_1614022440181/work
pathlib2 @ file:///C:/ci/pathlib2_1607025069150/work
pathspec==0.7.0
patsy==0.5.1
pep8==1.7.1
pexpect @ file:///tmp/build/80754af9/pexpect_1605563209008/work
pickleshare @ file:///tmp/build/80754af9/pickleshare_1606932040724/work
Pillow @ file:///C:/ci/pillow_1617386341487/work
pkginfo==1.7.0
plotly==5.2.2
pluggy @ file:///C:/ci/pluggy_1615976358795/work
ply==3.11
prometheus-client @ file:///tmp/build/80754af9/prometheus_client_1618088486455/work
prompt-toolkit @ file:///tmp/build/80754af9/prompt-toolkit_1616415428029/work
protobuf==3.19.1
psutil @ file:///C:/ci/psutil_1612298324802/work
ptyprocess @ file:///tmp/build/80754af9/ptyprocess_1609355006118/work/dist/ptyprocess-0.7.0-py2.py3-none-any.whl
py @ file:///tmp/build/80754af9/py_1607971587848/work
pyasn1==0.4.8
pyasn1-modules==0.2.8
pycodestyle @ file:///home/ktietz/src/ci_mi/pycodestyle_1612807597675/work
pycosat==0.6.3
pycparser @ file:///tmp/build/80754af9/pycparser_1594388511720/work
pycurl==7.43.0.6
pydocstyle @ file:///tmp/build/80754af9/pydocstyle_1616182067796/work
pyerfa @ file:///C:/ci/pyerfa_1619391071834/work
pyflakes @ file:///home/ktietz/src/ci_ipy2/pyflakes_1612551159640/work
Pygments @ file:///tmp/build/80754af9/pygments_1615143339740/work
pylint @ file:///C:/ci/pylint_1617136058775/work
pyls-black @ file:///tmp/build/80754af9/pyls-black_1607553132291/work
pyls-spyder @ file:///tmp/build/80754af9/pyls-spyder_1613849700860/work
PyNaCl @ file:///C:/ci/pynacl_1595000047588/work
pyodbc===4.0.0-unsupported
pyOpenSSL @ file:///tmp/build/80754af9/pyopenssl_1608057966937/work
pyparsing @ file:///home/linux1/recipes/ci/pyparsing_1610983426697/work
pyreadline==2.1
pyrsistent @ file:///C:/ci/pyrsistent_1600141795814/work
PySocks @ file:///C:/ci/pysocks_1605287845585/work
pytest==6.2.3
python-dateutil @ file:///home/ktietz/src/ci/python-dateutil_1611928101742/work
python-jsonrpc-server @ file:///tmp/build/80754af9/python-jsonrpc-server_1600278539111/work
python-language-server @ file:///tmp/build/80754af9/python-language-server_1607972495879/work
pytz @ file:///tmp/build/80754af9/pytz_1612215392582/work
PyWavelets @ file:///C:/ci/pywavelets_1601658407916/work
pywin32==227
pywin32-ctypes==0.2.0
pywinpty==0.5.7
PyYAML==5.4.1
pyzmq==20.0.0
QDarkStyle==2.8.1
QtAwesome @ file:///tmp/build/80754af9/qtawesome_1615991616277/work
qtconsole @ file:///tmp/build/80754af9/qtconsole_1616775094278/work
QtPy==1.9.0
regex @ file:///C:/ci/regex_1617569893741/work
requests @ file:///tmp/build/80754af9/requests_1608241421344/work
requests-oauthlib==1.3.0
rope @ file:///tmp/build/80754af9/rope_1602264064449/work
rsa==4.8
Rtree @ file:///C:/ci/rtree_1618421009405/work
ruamel-yaml-conda @ file:///C:/ci/ruamel_yaml_1616016967756/work
scikit-image==0.18.1
scikit-learn @ file:///C:/ci/scikit-learn_1614446896245/work
scipy @ file:///C:/ci/scipy_1618856128765/work
seaborn @ file:///tmp/build/80754af9/seaborn_1608578541026/work
Send2Trash @ file:///tmp/build/80754af9/send2trash_1607525499227/work
simplegeneric==0.8.1
singledispatch @ file:///tmp/build/80754af9/singledispatch_1614366001199/work
sip==4.19.13
six @ file:///C:/ci/six_1605187374963/work
sniffio @ file:///C:/ci/sniffio_1614030707456/work
snowballstemmer @ file:///tmp/build/80754af9/snowballstemmer_1611258885636/work
sortedcollections @ file:///tmp/build/80754af9/sortedcollections_1611172717284/work
sortedcontainers @ file:///tmp/build/80754af9/sortedcontainers_1606865132123/work
soupsieve @ file:///tmp/build/80754af9/soupsieve_1616183228191/work
speedtest-cli==2.1.3
Sphinx @ file:///tmp/build/80754af9/sphinx_1620777493457/work
sphinxcontrib-applehelp @ file:///home/ktietz/src/ci/sphinxcontrib-applehelp_1611920841464/work
sphinxcontrib-devhelp @ file:///home/ktietz/src/ci/sphinxcontrib-devhelp_1611920923094/work
sphinxcontrib-htmlhelp @ file:///home/ktietz/src/ci/sphinxcontrib-htmlhelp_1611920974801/work
sphinxcontrib-jsmath @ file:///home/ktietz/src/ci/sphinxcontrib-jsmath_1611920942228/work
sphinxcontrib-qthelp @ file:///home/ktietz/src/ci/sphinxcontrib-qthelp_1611921055322/work
sphinxcontrib-serializinghtml @ file:///home/ktietz/src/ci/sphinxcontrib-serializinghtml_1611920755253/work
sphinxcontrib-websupport @ file:///tmp/build/80754af9/sphinxcontrib-websupport_1597081412696/work
spyder @ file:///C:/ci/spyder_1616776239898/work
spyder-kernels @ file:///C:/ci/spyder-kernels_1614030842607/work
SQLAlchemy @ file:///C:/ci/sqlalchemy_1618090063261/work
statsmodels==0.12.2
sympy @ file:///C:/ci/sympy_1618255511605/work
tables==3.6.1
tblib @ file:///tmp/build/80754af9/tblib_1597928476713/work
tenacity==8.0.1
tensorboard==2.7.0
tensorboard-data-server==0.6.1
tensorboard-plugin-wit==1.8.1
tensorflow==2.7.0
tensorflow-estimator==2.7.0
tensorflow-io-gcs-filesystem==0.23.1
termcolor==1.1.0
terminado==0.9.4
testpath @ file:///home/ktietz/src/ci/testpath_1611930608132/work
textdistance @ file:///tmp/build/80754af9/textdistance_1612461398012/work
threadpoolctl @ file:///tmp/tmp9twdgx9k/threadpoolctl-2.1.0-py3-none-any.whl
three-merge @ file:///tmp/build/80754af9/three-merge_1607553261110/work
tifffile @ file:///tmp/build/80754af9/tifffile_1619636090847/work
toml @ file:///tmp/build/80754af9/toml_1616166611790/work
toolz @ file:///home/linux1/recipes/ci/toolz_1610987900194/work
tornado @ file:///C:/ci/tornado_1606942392901/work
tqdm @ file:///tmp/build/80754af9/tqdm_1615925068909/work
traitlets @ file:///home/ktietz/src/ci/traitlets_1611929699868/work
typed-ast @ file:///C:/ci/typed-ast_1610466535590/work
typing-extensions @ file:///home/ktietz/src/ci_mi/typing_extensions_1612808209620/work
ujson @ file:///C:/ci/ujson_1611241570789/work
unicodecsv==0.14.1
urllib3 @ file:///tmp/build/80754af9/urllib3_1615837158687/work
watchdog @ file:///C:/ci/watchdog_1612471251191/work
wcwidth @ file:///tmp/build/80754af9/wcwidth_1593447189090/work
webencodings==0.5.1
Werkzeug @ file:///home/ktietz/src/ci/werkzeug_1611932622770/work
wget==3.2
widgetsnbextension==3.5.1
win-inet-pton @ file:///C:/ci/win_inet_pton_1605306167264/work
win-unicode-console==0.5
wincertstore==0.2
wordcloud==1.8.1
wrapt==1.12.1
xlrd @ file:///tmp/build/80754af9/xlrd_1608072521494/work
XlsxWriter @ file:///tmp/build/80754af9/xlsxwriter_1617224712951/work
xlwings==0.23.0
xlwt==1.3.0
xmltodict==0.12.0
yapf @ file:///tmp/build/80754af9/yapf_1615749224965/work
zict==2.0.0
zipp @ file:///tmp/build/80754af9/zipp_1615904174917/work
zope.event==4.5.0
zope.interface @ file:///C:/ci/zope.interface_1616357322857/work